In [11]:
import pdfminer, pdfminer.high_level
import os, re
import pandas as pd

In [12]:
if not os.path.exists('done'):
    os.makedirs('done')

In [13]:
num_pages = 0
text = ''
file_names = []
for file in os.listdir('.'):
    if file.endswith('.pdf'):
        with open(file, 'rb') as f:
            text += pdfminer.high_level.extract_text(f)
        current_num_pages = len(list(pdfminer.high_level.extract_pages(file)))
        num_pages += current_num_pages
        file_names.extend([file] * current_num_pages)
        os.rename(file, 'done/' + file)

pages = text.split('\x0c')

In [14]:
pages
pages = [page for page in pages if page]

In [15]:
pages

['NUMER FAKTURY |\n155414781Z\n\nDATA TRANSAKCJI\n2024.07.24\n\n SZÁMLASZÁM\n\n|\n\n TELJESÍTÉS DÁTUMA\n\nDATA WYSTAWIENIA |\n2024.07.24\n\n SZÁMLA KELTE\n\n AZONOSÍTÓ\n\nPNR |\nALKSXH\n\nTERMIN PŁATNOŚCI\n2024.07.24\n\n|\n\n FIZETÉSI HATÁRIDŐ\n\nSTRONA |\n\n OLDAL 1 / 1\n\nNAZWA SPRZEDAWCY |\nWizz Air Hungary Légiközlekedési Zrt.\n\n SZÁLLÍTÓ NÉV\n\nODBIORCA |\nOREX TRAVEL Sp. Z o.o.\n\n VEVŐ NÉV\n\n CÍM\n\nADRES |\nBudapest\nLechner Ödön fasor 6.\n1095\n\n CÍM\n\nADRES |\nWarszawa\nAleje Jerozolimskie 132\n02-305\nPL\n\nEU VAT NUMER |\nHU26648525\n\n UNIÓS ADÓSZÁM\n\nNIP ODBIORCY |\n9542736185\n\n VEVŐ ADÓSZÁM\n\nVAT NUMER |\n26648525-2-44\n\n ADÓSZÁM\n\nILOŚĆ\n\nMENNY.\n\n1,00\n\nJEDNOSTKA MIARY\n\nCENA JEDN.\n\nMÉRTÉKEGYSÉG\n\nEGYSÉGÁR\n\nVAT%\n\nÁFA%\n\nKWOTA NETTO\n\nVAT\n\nKWOTA BRUTTO\n\nWALUTA\n\nÉRTÉKE ÁFA NÉLKÜL\n\nÁTHÁRÍTOTT ÁFA\n\nÉRTÉKE ÁFÁVAL\n\nPÉNZNEM\n\nSztuk / db\n\n-20,00\n\n0.00 %\n\n-20,00\n\n0,00\n\n-20,00\n\nEUR\n\nRODZAJ USŁUGI\nBilet lotniczy / Repülőjegy (BCN

In [16]:
NUMER = r'NUMER FAKTURY.*?\n(.*?)\n'
DATA = r'DATA WYSTAWIENIA.*?\n(.*?)\n'
KWOTA = re.compile(r'PODSUMOWANIE.*WALUTA.*\n(.*?\,.*?)\n.*?\,.*EUR', re.DOTALL)

In [17]:
faktury = [['numer', 'data', 'kwota']]
for page in pages:
    numer = re.search(NUMER, page).group(1)
    data = re.search(DATA, page).group(1)
    kwota = re.search(KWOTA, page).group(1)
    faktury.append([numer, data, kwota])

In [18]:
faktury

[['numer', 'data', 'kwota'],
 ['155414781Z', '2024.07.24', '-20,00'],
 ['155414852Z', '2024.07.24', '-16,00'],
 ['155414980Z', '2024.07.24', '-90,00'],
 ['155415023Z', '2024.07.24', '-364,00'],
 ['155415128Z', '2024.07.24', '-80,00'],
 ['155415183Z', '2024.07.24', '-120,00'],
 ['155415195Z', '2024.07.24', '-290,00'],
 ['155415198Z', '2024.07.24', '-80,00'],
 ['155415280Z', '2024.07.24', '-10,00'],
 ['155415363Z', '2024.07.24', '-36,00'],
 ['155415400Z', '2024.07.24', '-64,00'],
 ['155415405Z', '2024.07.24', '-10,00'],
 ['155415565Z', '2024.07.24', '-4,00'],
 ['155415614Z', '2024.07.24', '-48,00'],
 ['155415900Z', '2024.07.24', '-13,00'],
 ['155415954Z', '2024.07.24', '-140,00'],
 ['155415962Z', '2024.07.24', '-10,00'],
 ['155416035Z', '2024.07.24', '-13,00'],
 ['155416105Z', '2024.07.24', '-20,00'],
 ['155416204Z', '2024.07.24', '-40,00'],
 ['155416236Z', '2024.07.24', '-12,00'],
 ['155416324Z', '2024.07.24', '-10,00'],
 ['155416519Z', '2024.07.24', '-390,00'],
 ['155416525Z', '2024.07

In [19]:
faktury = pd.DataFrame(faktury[1:], columns=faktury[0])

In [20]:
faktury.to_excel('faktury.xlsx', index=False)